In [16]:
from ultralytics import YOLO
import cv2

# Load the YOLO model
model = YOLO('fire.pt')

In [17]:
# Define the data
data = {
    'degree': [55],
    'd1': [d_values[0],d_values[1],d_values[2],d_values[3],d_values[4]]
}

# Define the value for num
num = 1

# Function to determine the y-coordinate of the black line based on d1 value
def get_black_line_y(d1_value, green_lines, frame_height):
    if d1_value == 0:
        return frame_height  # Black line at the bottom
    if d1_value < green_lines[0][1]:
        return frame_height - int((green_lines[0][0] - (frame_height - d1_value * (green_lines[0][0] - green_lines[-1][0]) / green_lines[0][1])) - frame_height)
    for i in range(len(green_lines) - 1):
        if green_lines[i][1] < d1_value <= green_lines[i + 1][1]:
            return green_lines[i][0] - int((green_lines[i][0] - green_lines[i + 1][0]) * (d1_value - green_lines[i][1]) / (green_lines[i + 1][1] - green_lines[i][1]))
    return green_lines[0][0]  # Above the highest green line

# Function to determine the base line label based on black lines
def get_base_line_label(y_bottom, black_lines, num, opposite):
    # Ensure 'opposite' is a valid float value
    try:
        opposite = float(opposite)  # Ensure it's a float
    except (ValueError, TypeError):
        opposite = 0.0  # Default to 0.0 if not a valid number

    # Sort black lines by y-coordinate
    black_lines = sorted(black_lines, key=lambda x: x[0], reverse=True)

    # Determine the range
    for i in range(len(black_lines) - 1):
        if black_lines[i + 1][0] < y_bottom <= black_lines[i][0]:
            return f"Base line {(i + 1) * num + opposite:.2f} to {(i + 2) * num + opposite:.2f}"
    
    # Special cases
    if y_bottom > black_lines[0][0]:
        return f"Base line 0 to {num + opposite:.2f}"
    if y_bottom <= black_lines[-1][0]:
        return f"Base line {len(black_lines) * num + opposite:.2f} to top"
    
    # Default case if none of the above conditions are met
    return f"Base line 0 to {num + opposite:.2f}"